In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
pd.set_option('display.precision',10)

C:\Users\anuku\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Changes made are:

a) Internal Intersection Type 1 and 4 changes 

b) Manual changes for 2 links 

c) Adding 'RAMP' attribute to the links file 

In [ ]:
#LINKS combined from Here - 3Million
all_links = pd.read_csv('../midstages/all_links.csv')
all_links_f = all_links[['LINK_ID','ST_NAME','REF_IN_ID',
 'NREF_IN_ID',
 'N_SHAPEPNT',
 'FUNC_CLASS',
 'SPEED_CAT',
 'FR_SPD_LIM',
 'TO_SPD_LIM',
 'TO_LANES',
 'FROM_LANES',
 'ENH_GEOM',
 'LANE_CAT',
 'DIVIDER',
 'DIR_TRAVEL','PHYS_LANES','INTERINTER', 'RAMP']]
 #partner links
partner_links = pd.read_csv("../final/partner_links_thrufilter.csv")

### EDA

In [ ]:
#Internal intersection links
internalinter = all_links_f[all_links_f['INTERINTER']=='Y']
len(internalinter)

In [ ]:
internalinter.groupby(['DIR_TRAVEL'])['LINK_ID'].count()

In [ ]:
internalinter.groupby(['PHYS_LANES'])['LINK_ID'].count()

In [ ]:
internalinter.groupby(['PHYS_LANES','DIR_TRAVEL'])['LINK_ID'].count()

In [ ]:
internalinter.groupby('RAMP')['LINK_ID'].count() # 244 RAMPS

In [ ]:
internalinter[internalinter["RAMP"]=='Y'].groupby(['PHYS_LANES'])['LINK_ID'].count()

In [ ]:
internalinter.SPEED_CAT.value_counts()

### 1.  Type 1 Internal Intersection

#### PHYS LANE = 1 & Unidirectional

In [ ]:
#Internat intersction links, Phy lanes 1
internalinter1 = internalinter[internalinter['PHYS_LANES']==1]
print(len(internalinter1))

In [ ]:
#RAMP are mostly 1 lane 
internalinter1[internalinter1['RAMP']=='Y']['TO_LANES'].value_counts()

In [ ]:
internalinter1[internalinter1['RAMP']=='Y']['FROM_LANES'].value_counts()

In [ ]:
## SOLUTION FOR TYPE 1 - STEP1 - TAKE VALUES from TO_LANES or FROM_LANES

In [ ]:
# Since we found that if both from and to lanes are 0, ad dthe phy lanes as 1

def inter_normall(row):
    if row['TO_LANES'] != 0:
        row['sp_lanes'] = row['TO_LANES']
    elif row['FROM_LANES'] != 0:
        row['sp_lanes'] = row['FROM_LANES']
    else:
        row['sp_lanes'] = 1
    return row['sp_lanes']

In [ ]:
internalinter1['sp_lanes'] = 0
internalinter1['sp_lanes'] = internalinter1.apply(lambda row:inter_normall(row), axis=1)

In [ ]:
internalinter1.groupby(['PHYS_LANES','sp_lanes'])['LINK_ID'].count() # total of 6,509 links 

In [ ]:
len(internalinter1)

#### Apply (August 2020 Update)

In [ ]:
ca= pd.read_csv("../Nov2019/for_drive/all_links_noferry_thrufilter.csv")
len(ca)

In [ ]:
ca1 = ca.merge(internalinter1[['LINK_ID','sp_lanes']], left_on = 'LINK_ID', right_on = 'LINK_ID', how = 'left')
print(len(ca1))
ca1.head(2)

In [ ]:
ca1['py'] = np.where(pd.isnull(ca1['sp_lanes']),ca1['NUM_PHYS_LANES'],ca1['sp_lanes'])

In [ ]:
print(len(ca1[ca1['py'] != ca1['NUM_PHYS_LANES']])) #actual change, some II links still has same1

In [ ]:
ca1 = ca1[['LINK_ID', 'ST_NAME', 'REF_IN_ID', 'NREF_IN_ID', 'FUNC_CLASS',
       'DIR_TRAVEL', 'SPEED_KPH', 'CAPACITY(veh/hour)',
       'LENGTH(meters)', 'N_SHAPEPNT','py']]
ca1.rename(columns = {'py':'NUM_PHYS_LANES'}, inplace = True)
ca1 = ca1[['LINK_ID', 'ST_NAME', 'REF_IN_ID', 'NREF_IN_ID', 'FUNC_CLASS',
       'DIR_TRAVEL', 'NUM_PHYS_LANES', 'SPEED_KPH', 'CAPACITY(veh/hour)',
       'LENGTH(meters)', 'N_SHAPEPNT']]

In [ ]:
# Modify Capacity

In [ ]:
all_nolen = pd.read_csv("../internal_intersection/all_links_ca_uni_nolen.csv")
all_nolen.head(2) # has the speed category

In [ ]:
# adding speed category column
ca1= ca1.merge(all_nolen[['LINK_ID','SPEED_CAT']], left_on = 'LINK_ID', right_on = 'LINK_ID', how = 'left')
ca1.head(2)

In [ ]:
#4.calculate capacities based on speed categories and not based on functional class ((values per lane is as per matsim))
#veh/hour
def capacity(row):
    if row['SPEED_CAT'] ==1:
        return 2000
    if row['SPEED_CAT'] ==2:
        return 2000
    if row['SPEED_CAT'] ==3:
        return 2000
    if row['SPEED_CAT'] ==4:
        return 1500
    if row['SPEED_CAT'] ==5:
        return 1500
    if row['SPEED_CAT'] ==6:
        return 1000
    if row['SPEED_CAT'] ==7:
        return 600
    if row['SPEED_CAT'] ==8:
        return 300

In [ ]:
#Total capacity
ca1['CAPACITY'] = ca1.apply(lambda row:capacity(row), axis=1)
ca1['CAPACITY'] = ca1['CAPACITY'] * ca1['NUM_PHYS_LANES']

In [ ]:
#chnaged
len(all_links1[all_links1['CAPACITY(veh/hour)']!=all_links1['CAPACITY'] ])  

### 2. Type 4 Internal Intersection

The original link ids get FROM_LANES values, partners get FROM_LANE values

In [ ]:
#example
all_links_f[all_links_f['LINK_ID'].isin([7000262387,816272333])] # should be 2 for link and its partner but lane dbf assigns just 1

In [ ]:
internalinter4 = internalinter[(internalinter['PHYS_LANES']>1) & (internalinter['DIR_TRAVEL']>='B')]
len(internalinter4) # all links, not filetred

In [ ]:
partner_links_type4 = partner_links[partner_links['LINK_ID'].isin(internalinter4.LINK_ID)]
len(partner_links_type4) #these are the ones after thry filter which we care about 

In [ ]:
partner_links_type4_t = partner_links_type4.merge(internalinter4[['LINK_ID','TO_LANES','FROM_LANES']], left_on = 'LINK_ID',right_on = 'LINK_ID', how = 'left')
len(partner_links_type4_t)

In [ ]:
t1 = partner_links_type4_t[['LINK_ID', 'FROM_LANES']]
t2 = partner_links_type4_t[['partner_LINK_ID', 'TO_LANES']]
t1.rename(columns = {'FROM_LANES':'sp_values'}, inplace = True)
t2.rename(columns = {'partner_LINK_ID':'LINK_ID','TO_LANES':'sp_values'}, inplace = True)

In [ ]:
t3 = pd.concat([t1,t2])
len(t3)

In [ ]:
t3 = t3.merge(l[l['LINK_ID'].isin(t3['LINK_ID'])][['LINK_ID', 'NUM_PHYS_LANES']], left_on = 'LINK_ID',right_on = 'LINK_ID', how = 'left')

In [ ]:
# if spvalue is bigger that is assigned, else lane.dbf file value is kept
t3['new_value'] = np.where(t3['sp_values']>t3['NUM_PHYS_LANES'], t3['sp_values'], t3['NUM_PHYS_LANES'])
t3.head(10)

In [ ]:
t3[t3['LINK_ID'].isin([7000262387,816272333])] # so for these 10544 links 'new_value' is the new phys_lane values

In [ ]:
print(len(t3[t3['new_value'] != t3['NUM_PHYS_LANES']])) #actual change, some II links still has same1

#### Apply


In [ ]:
# august all links file which has type1 chnage already implemented

ca= pd.read_csv("../Nov2019/for_drive/august_2020/all_links.csv")
len(ca)

In [ ]:
ca1 = ca.merge(t3[['LINK_ID','new_value']], left_on = 'LINK_ID', right_on = 'LINK_ID', how = 'left')
print(len(ca1))
ca1.head(2)

In [ ]:
ca1['py'] = np.where(pd.isnull(ca1['new_value']),ca1['NUM_PHYS_LANES'],ca1['new_value'])

In [ ]:
print(len(ca1[ca1['py'] != ca1['NUM_PHYS_LANES']])) #actual change, some II links still has same1

In [ ]:
ca1 = ca1[['LINK_ID', 'ST_NAME', 'REF_IN_ID', 'NREF_IN_ID', 'FUNC_CLASS',
       'DIR_TRAVEL', 'SPEED_KPH', 'CAPACITY(veh/hour)',
       'LENGTH(meters)', 'N_SHAPEPNT','py']]
ca1.rename(columns = {'py':'NUM_PHYS_LANES'}, inplace = True)
ca1 = ca1[['LINK_ID', 'ST_NAME', 'REF_IN_ID', 'NREF_IN_ID', 'FUNC_CLASS',
       'DIR_TRAVEL', 'NUM_PHYS_LANES', 'SPEED_KPH', 'CAPACITY(veh/hour)',
       'LENGTH(meters)', 'N_SHAPEPNT']]

### 3. Manual Changes for 2 links

I-280

Jackson street 

In [ ]:
ca1[ca1.LINK_ID.isin([782677616,7000072004])]

In [ ]:
ca1.loc[2126570, 'NUM_PHYS_LANES'] = 2
ca1.loc[4313592, 'NUM_PHYS_LANES'] = 6

In [ ]:
ca1[ca1.LINK_ID.isin([782677616,7000072004])]

#### Application - Capacity Changes

In [ ]:
# adding speed category column
ca1 = ca1.merge(all_nolen[['LINK_ID','SPEED_CAT']], left_on = 'LINK_ID', right_on = 'LINK_ID', how = 'left')
len(ca1)

In [ ]:
#Total capacity
ca1['CAPACITY'] = ca1.apply(lambda row:capacity(row), axis=1)
ca1['CAPACITY'] = ca1['CAPACITY'] * ca1['NUM_PHYS_LANES']

In [ ]:
#chnaged
len(ca1[ca1['CAPACITY(veh/hour)']!=ca1['CAPACITY'] ])

In [ ]:
change = ca1[ca1['CAPACITY(veh/hour)']!=ca1['CAPACITY'] ]
change.groupby(['CAPACITY(veh/hour)','CAPACITY'])['LINK_ID'].count()

In [ ]:
ca_f = ca1[['LINK_ID', 'ST_NAME', 'REF_IN_ID', 'NREF_IN_ID', 'FUNC_CLASS',
       'DIR_TRAVEL', 'NUM_PHYS_LANES', 'SPEED_KPH', 'LENGTH(meters)', 'CAPACITY']]
ca_f.rename(columns = {'CAPACITY':'CAPACITY(veh/hour)'}, inplace = True)

In [ ]:
# Manual changed links
ca_f[ca_f.LINK_ID.isin([782677616,7000072004])]

### 4. Adding Ramp feature to Links file

In [ ]:
# For CA
# selecting calinks in streets 
sfstreets = all_links[all_links['LINK_ID'].isin(ca_f['LINK_ID'])]
#check how many bidirectional in these 
print("Type in streets", sfstreets.groupby(['RAMP', 'DIR_TRAVEL'])['LINK_ID'].count())

# ALL CA links which are ramps
#uni
sfstreetsrampunif = sfstreets[(sfstreets['RAMP'] == 'Y') & (sfstreets['DIR_TRAVEL'] == 'F')]['LINK_ID'].to_list() #sflinks ramp f
sfstreetsrampunit = sfstreets[(sfstreets['RAMP'] == 'Y') & (sfstreets['DIR_TRAVEL'] == 'T')]['LINK_ID'].to_list() #sflinks ramp t
#bi
sfstreetsrampbi = sfstreets[(sfstreets['RAMP'] == 'Y') & (sfstreets['DIR_TRAVEL'] == 'B')]['LINK_ID'].to_list() #sflinks ramp bi
sfstreetsrampbi_partner = partner_links[partner_links['LINK_ID'].isin(sfstreetsrampbi)]['partner_LINK_ID'].to_list() #sflinks ram bi partners


print("Unidirectional number of ramps", len(sfstreetsrampunif)+len(sfstreetsrampunit)+ len(sfstreetsrampbi)+len(sfstreetsrampbi_partner)) # len of 

#Adding new column in sf links which have ramp
ca_f['RAMP'] = 'N'
rampsf = [sfstreetsrampunif,sfstreetsrampunit,sfstreetsrampbi,sfstreetsrampbi_partner]
for i in rampsf:
    ca_f.loc[ca_f.LINK_ID.isin(i),'RAMP'] = 'Y'
ca_f['RAMP'].value_counts()

### 5.Clip SF and LA

In [ ]:
oldsf = pd.read_csv("../Nov2019/for_drive/august_2020/sf_links.csv")
oldla = pd.read_csv("../Nov2019/for_drive/august_2020/la_links.csv")
len(oldsf), len(oldla)

In [ ]:
sf = ca_f[ca_f.LINK_ID.isin(oldsf.LINK_ID)]
len(sf)

In [ ]:
la = ca_f[ca_f.LINK_ID.isin(oldla.LINK_ID)]
len(la)

In [ ]:
ca_f.to_csv("../Nov2019/for_drive/september2020/all_links.csv", index = False)
sf.to_csv("../Nov2019/for_drive/september2020/sf_links.csv", index = False)
la.to_csv("../Nov2019/for_drive/september2020/la_links.csv", index = False)